In [25]:
# !pip install selenium

In [26]:
'''
1.需要提前打开手机12306二维码扫码工具
2.默认选择靠近过道的座位，若座位过少无法支持选座则该步操作忽略
3.仅支持购买二等座票
4.默认购买排序第一个人的票
'''

'\n1.需要提前打开手机12306二维码扫码工具\n2.默认选择靠近过道的座位，若座位过少无法支持选座则该步操作忽略\n3.仅支持购买二等座票\n4.默认购买排序第一个人的票\n'

In [27]:
from pathlib import Path
import json
from datetime import datetime


def load_config(config_path: Path = Path('config.json')):
    try:
        if config_path.exists():
            with config_path.open('r', encoding='utf-8') as f:
                return json.load(f)
    except Exception as e:
        print(f'读取配置失败: {e}')
    return None


def save_config(config: dict, config_path: Path = Path('config.json')):
    try:
        with config_path.open('w', encoding='utf-8') as f:
            json.dump(config, f, ensure_ascii=False, indent=2)
        print(f'已保存配置到 {config_path.resolve()}')
    except Exception as e:
        print(f'保存配置失败: {e}')


def _ask(prompt_text: str, default: str = ''):
    s = input(f"{prompt_text} [{default}]: ").strip()
    return s or default


def _ask_list(prompt_text: str, default_list):
    s = input(f"{prompt_text}（逗号分隔）[{','.join(default_list)}]: ").strip()
    return [x.strip() for x in (s.split(',') if s else default_list) if x.strip()]


def get_params():
    cfg = load_config()
    use_cfg = None
    if cfg:
        ans = input('检测到 config.json，是否使用该配置? (Y/n): ').strip().lower()
        use_cfg = (ans == '' or ans == 'y' or ans == 'yes')
    if cfg and use_cfg:
        return cfg

    # 交互式输入
    start_position = _ask('出发站(需完整名称)', '广州南')
    reach_position = _ask('到达站(需完整名称)', '上海南')
    start_date = _ask('出发日期(YYYY-MM-DD)', datetime.now().strftime('%Y-%m-%d'))
    train_number = _ask('目标车次(如 G818)', 'G818')
    ticket_type = _ask('票型 adult(成人)/student(学生)', 'adult').lower()
    booking_start_time = _ask('开售时间(YYYY-MM-DD HH:MM:SS)', (datetime.now().strftime('%Y-%m-%d') + ' 15:00:00'))
    seat_class_priority = _ask_list('席别优先顺序', ['二等座', '一等座', '商务座'])
    seat_preference = _ask('选座偏好(过道/靠窗/随机)', '过道')
    allow_reserve = _ask('无票时是否候补? (Y/n)', 'Y').strip().lower() in ('y', 'yes')
    max_attempts = int(_ask('最大重试次数', '30'))
    refresh_min = float(_ask('刷新最小间隔(秒)', '0.5'))
    refresh_max = float(_ask('刷新最大间隔(秒)', '1.5'))
    burst_enable = _ask('开售瞬间连点查询? (Y/n)', 'Y').strip().lower() in ('y', 'yes')
    burst_duration = float(_ask('连点持续时间(秒)', '2.0'))
    burst_min = float(_ask('连点最小间隔(秒)', '0.08'))
    burst_max = float(_ask('连点最大间隔(秒)', '0.16'))
    warmup_seconds = float(_ask('开售前预热秒数(提前就绪)', '10'))

    params = {
        "start_position": start_position,
        "reach_position": reach_position,
        "start_date": start_date,
        "ticket_type": "student" if ticket_type.startswith('s') else "adult",
        "target_train_number": train_number,
        "booking_start_time": booking_start_time,
        "seat_class_priority": seat_class_priority,
        "seat_preference": seat_preference,
        "allow_reserve": allow_reserve,
        "retry": {"max_attempts": max_attempts, "refresh_interval": [refresh_min, refresh_max]},
        "burst": {"enable": burst_enable, "duration_sec": burst_duration, "min_interval": burst_min, "max_interval": burst_max},
        "warmup_seconds": warmup_seconds,
    }

    try_save = input('是否保存为 config.json 以便下次直接使用? (Y/n): ').strip().lower()
    if try_save in ('', 'y', 'yes'):
        save_config(params)
    return params

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from datetime import datetime

def retry_book_ticket(driver, target_train_number, max_attempts=30, refresh_interval=(0.5, 1.5)):
    for attempt in range(1, max_attempts + 1):
        try:
            WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.ID, "queryLeftTable"))
            )
            book_buttons = driver.find_elements(By.XPATH,
                                                f"//tr[contains(., '{target_train_number}')]//a[contains(text(), '预订')]")

            if book_buttons:
                book_button = book_buttons[0]

            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center', inline: 'center'});",
                book_button
            )
            time.sleep(3)

            try:
                book_button.click()

                return f"成功预订{target_train_number}车次"
            except Exception as e:

                driver.execute_script("arguments[0].click();", book_button)

                return f"成功预订{target_train_number}车次"

        except Exception as e:
            print(f"第{attempt}次尝试失败: {str(e)}")

            if attempt < max_attempts:
                try:
                    refresh_btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.ID, "query_ticket"))
                    )
                    refresh_btn.click()

                    wait_time = random.uniform(*refresh_interval)
                    print(f"等待{wait_time:.2f}秒后进行下一次尝试...")
                    time.sleep(wait_time)

                except Exception as refresh_err:
                    print(f"刷新页面失败: {str(refresh_err)}，尝试直接刷新页面...")
                    driver.refresh()
                    time.sleep(random.uniform(*refresh_interval))

    return f"没抢到，可惜~"

def select_seat_dynamically(driver, preferred_type="过道"):
    print(f"开始选择座位，首选类型: {preferred_type}")

 
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "seat-sel-bd"))
        )
        print("座位选择对话框已加载")
        time.sleep(1) 
    except Exception as e:
        print(f"座位选择对话框加载失败: {str(e)}")
        return False

 
    seat_type_mapping = {
        "靠窗": "窗",
        "过道": "过道"
    }

    if preferred_type == "靠窗":
        try_order = ["靠窗", "过道", "随机"]
    else:
        try_order = ["过道", "靠窗", "随机"]

    for seat_type in try_order:
        try:
            print(f"尝试选择【{seat_type}】座位...")

            if seat_type == "随机":
 
                available_seats = driver.find_elements(By.XPATH,
                                                       "//ul[@class='seat-list']//a[contains(@href, 'javascript:')]")
                if available_seats:
                    available_seats[0].click()
                    print("随机选择了一个可用座位")
                    return True
                else:
                    print("没有找到可用座位")
                    continue

            target_text = seat_type_mapping[seat_type]

            text_elements = driver.find_elements(By.XPATH, f"//div[@class='txt' and contains(text(), '{target_text}')]")

            if not text_elements:
                print(f"未找到'{target_text}'标识")
                continue
            for text_element in text_elements:
                try:
 
                    seat_list = text_element.find_element(By.XPATH, "./following-sibling::ul[@class='seat-list']")

 
                    seat_buttons = seat_list.find_elements(By.XPATH, ".//a[contains(@href, 'javascript:')]")

                    if seat_buttons:
 
                        seat_buttons[0].click()
                        print(f"成功选择【{seat_type}】座位")
                        return True

                except Exception as e:
                    print(f"在处理'{target_text}'区域时出错: {str(e)}")
                    continue

        except Exception as e:
            print(f"选择【{seat_type}】座位失败：{str(e)}")
            continue
    try:
        all_seat_buttons = driver.find_elements(By.XPATH, "//ul[@class='seat-list']//a[contains(@href, 'javascript:')]")
        if all_seat_buttons:
            all_seat_buttons[0].click()
            print("成功选择了一个可用座位（全局查找）")
            return True
        else:
            print("全局查找也未找到可用座位")
    except Exception as e:
        print(f"全局查找座位失败：{str(e)}")

    return False

In [ ]:
# --- 更稳健的等待/点击工具与元素定位 ---
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys


def wait_for(driver, condition, timeout=15, poll_frequency=0.2):
    return WebDriverWait(driver, timeout, poll_frequency=poll_frequency).until(condition)


def safe_click(driver, locator, by=By.CSS_SELECTOR, timeout=10):
    try:
        el = wait_for(driver, EC.element_to_be_clickable((by, locator)), timeout)
        el.click()
        return True
    except Exception:
        try:
            el = driver.find_element(by, locator)
            driver.execute_script('arguments[0].click();', el)
            return True
        except Exception as e:
            print(f'safe_click 失败: {locator} -> {e}')
            return False


def safe_send_keys(driver, locator, text, by=By.CSS_SELECTOR, timeout=10, clear_first=True):
    try:
        el = wait_for(driver, EC.element_to_be_clickable((by, locator)), timeout)
        if clear_first:
            try:
                el.clear()
            except Exception:
                pass
        el.send_keys(text)
        return True
    except Exception as e:
        print(f'safe_send_keys 失败: {locator} -> {e}')
        return False


def precisely_wait_until(target_dt: datetime, warmup_seconds: float = 10.0):
    import time
    now = datetime.now()
    if now < target_dt:
        remain = (target_dt - now).total_seconds()
        print(f'距离目标时间还有 {remain:.1f}s，进入等待（预热 {warmup_seconds}s）...')
        if remain > warmup_seconds:
            time.sleep(remain - warmup_seconds)
        # 毫秒级忙等
        while datetime.now() < target_dt:
            time.sleep(0.001)
    print('已到目标时间')


def click_query_burst(driver, duration_sec=2.0, min_interval=0.08, max_interval=0.16):
    import random, time
    end_t = time.time() + duration_sec
    clicked = 0
    while time.time() < end_t:
        try:
            btn = WebDriverWait(driver, 0.5).until(EC.element_to_be_clickable((By.ID, 'query_ticket')))
            btn.click()
            clicked += 1
        except Exception:
            pass
        time.sleep(random.uniform(min_interval, max_interval))
    print(f'连点查询完成，次数={clicked}')


def js_dispatch_click(driver, el):
    driver.execute_script(
        "var e=document.createEvent('MouseEvents'); e.initEvent('click',true,true); arguments[0].dispatchEvent(e);",
        el,
    )


def robust_click_element(driver, el):
    # 滚动到视口中央并偏移，避免被顶部浮层遮挡
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    try:
        driver.execute_script("window.scrollBy(0,-80);")
    except Exception:
        pass

    # 多策略点击：原生 -> ActionChains -> JS click -> JS dispatch -> 键盘回车
    try:
        el.click()
        return True
    except Exception:
        try:
            ActionChains(driver).move_to_element(el).pause(0.05).click().perform()
            return True
        except Exception:
            try:
                driver.execute_script('arguments[0].click();', el)
                return True
            except Exception:
                try:
                    js_dispatch_click(driver, el)
                    return True
                except Exception:
                    try:
                        el.send_keys(Keys.ENTER)
                        return True
                    except Exception as e:
                        print(f'robust_click_element 失败: {e}')
                        return False


def take_debug_snapshot(driver, tag_prefix='debug'):
    import time
    ts = time.strftime('%Y%m%d_%H%M%S')
    png = f'{tag_prefix}_{ts}.png'
    html = f'{tag_prefix}_{ts}.html'
    try:
        driver.save_screenshot(png)
        with open(html, 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print(f'已保存快照: {png}, {html}')
    except Exception as e:
        print(f'保存快照失败: {e}')


# --- 结果加载等待与行定位 ---

def wait_results_ready(driver, timeout=10):
    import time
    try:
        wait_for(driver, EC.presence_of_element_located((By.ID, 'queryLeftTable')), timeout=timeout)
    except Exception:
        return False
    # 等待出现至少一行结果，最多等待 timeout 秒
    end_t = time.time() + timeout
    last_count = -1
    while time.time() < end_t:
        try:
            rows = driver.find_elements(By.XPATH, "//*[@id='queryLeftTable']//tr[not(contains(@class,'datatitle')) and not(contains(@style,'display: none'))]")
            if rows:
                if len(rows) == last_count:
                    return True
                last_count = len(rows)
        except Exception:
            pass
        time.sleep(0.2)
    return False


def _normalize_text(s: str) -> str:
    return (s or '').strip().replace('\u200b', '').replace('\xa0', '').replace(' ', '')


def find_train_row(driver, train_no: str):
    train_no_norm = _normalize_text(train_no)

    # 1) 先在 queryLeftTable 容器内查找
    try:
        container = driver.find_element(By.ID, 'queryLeftTable')
    except Exception:
        container = None

    rows = []
    if container:
        try:
            rows = container.find_elements(By.XPATH, ".//tr[not(contains(@class,'datatitle')) and not(contains(@style,'display: none'))]")
        except Exception:
            rows = []

    # 先通过 a.number 快速匹配
    if rows:
        for r in rows:
            try:
                a_nums = r.find_elements(By.XPATH, ".//a[contains(@class,'number')]")
                for a in a_nums:
                    if train_no_norm in _normalize_text(a.text):
                        return r
                # 退而求其次：整行文本匹配
                if train_no_norm in _normalize_text(r.text):
                    return r
            except Exception:
                continue

    # 2) 全局兜底：先按 a.number，再按任意文本包含
    try:
        els = driver.find_elements(By.XPATH, f"//a[contains(@class,'number')][contains(normalize-space(.), '{train_no}')]/ancestor::tr[1]")
        if els:
            return els[0]
    except Exception:
        pass
    try:
        els = driver.find_elements(By.XPATH, f"//tr[.//*[contains(normalize-space(.), '{train_no}')]]")
        if els:
            return els[0]
    except Exception:
        pass

    return None


# --- 席别映射与解析工具 ---
SEAT_CLASS_ALIASES = {
    '二等座': ['二等座', '二等', '二等椅', '二等票', '二等'],
    '一等座': ['一等座', '一等', '一等椅', '一等票', '一等'],
    '商务座': ['商务座', '商务', '商'],
    '特等座': ['特等座', '特等'],
    '无座': ['无座'],
    '硬座': ['硬座'],
    '软座': ['软座'],
}


def normalize_seat_name(name: str):
    for k, vs in SEAT_CLASS_ALIASES.items():
        for v in vs:
            if v in name:
                return k
    return name


def parse_left_ticket_text(txt: str):
    # 常见："有"、"无"、"候补"、"5"（数字）
    s = (txt or '').strip()
    if not s:
        return 0
    if '候补' in s:
        return '候补'
    if s == '有':
        return 9999
    try:
        return int(s)
    except Exception:
        return 0


def get_seat_cell_map(row_el):
    # 将行内各席别与对应单元格建立映射（基于列头文本顺序可能变化时，可适当增强）
    cells = row_el.find_elements(By.XPATH, ".//td")
    mapping = {}
    for td in cells:
        try:
            title = td.get_attribute('title') or td.get_attribute('data-title') or ''
            txt = (title or td.text or '').strip()
            if not txt:
                continue
            norm = normalize_seat_name(txt)
            mapping[norm] = td
        except Exception:
            pass
    return mapping


def click_book_or_reserve_in_row(row_el, prefer_reserve=False):
    # 优先点“预订”，若没找到且允许候补，点“候补”
    try:
        book = row_el.find_elements(By.XPATH, ".//a[contains(text(),'预订') and not(contains(@class,'btn-disabled'))]")
        if book:
            btn = book[0]
            try:
                row_el.parent.execute_script("arguments[0].scrollIntoView({block:'center'});", row_el)
            except Exception:
                pass
            if robust_click_element(row_el.parent, btn):
                return '预订'
        if prefer_reserve:
            hb = row_el.find_elements(By.XPATH, ".//a[contains(text(),'候补') and not(contains(@class,'btn-disabled'))]")
            if hb:
                if robust_click_element(row_el.parent, hb[0]):
                    return '候补'
    except Exception as e:
        print(f'点击预订/候补失败: {e}')
    return None


def retry_book_ticket_with_filters(driver, train_no: str, seat_priority: list, allow_reserve: bool, max_attempts=30, refresh_interval=(0.5, 1.5)):
    import random, time
    for attempt in range(1, max_attempts + 1):
        try:
            results_ready = wait_results_ready(driver, timeout=8)
            if not results_ready:
                print('结果容器未就绪，继续尝试…')
            row = find_train_row(driver, train_no)
            if row is None:
                raise TimeoutException('未找到目标车次行')

            seat_map = get_seat_cell_map(row)
            ok_to_book = False
            chosen_mode = None

            # 若席别映射为空，兜底直接尝试点“预订”
            if not seat_map:
                print('未能识别席别列，执行兜底点击预订。')
                chosen_mode = click_book_or_reserve_in_row(row, prefer_reserve=allow_reserve)
                if chosen_mode:
                    return f"成功触发{chosen_mode}，车次 {train_no}"

            for seat_name in seat_priority:
                td = None
                # 尝试按规范名匹配，或模糊匹配
                td = seat_map.get(seat_name)
                if td is None:
                    for k, v in seat_map.items():
                        if seat_name in k:
                            td = v
                            break
                if td is None:
                    continue
                left_txt = (td.text or td.get_attribute('textContent') or '').strip()
                left = parse_left_ticket_text(left_txt)
                print(f"席别[{seat_name}] 余票显示: '{left_txt}' -> {left}")
                if left == '候补' and allow_reserve:
                    chosen_mode = click_book_or_reserve_in_row(row, prefer_reserve=True)
                    ok_to_book = chosen_mode is not None
                    break
                if isinstance(left, int) and left > 0:
                    chosen_mode = click_book_or_reserve_in_row(row, prefer_reserve=False)
                    ok_to_book = chosen_mode is not None
                    break

            if ok_to_book or chosen_mode:
                return f"成功触发{chosen_mode or '预订'}，车次 {train_no}"

            print(f"第{attempt}次：席别均无票（或不候补），刷新重试…")
            if attempt < max_attempts:
                if safe_click(driver, 'query_ticket', by=By.ID, timeout=5):
                    time.sleep(random.uniform(*refresh_interval))
                else:
                    driver.refresh()
                    time.sleep(random.uniform(*refresh_interval))
        except Exception as e:
            print(f"第{attempt}次尝试失败: {e}")
            take_debug_snapshot(driver, tag_prefix='click_fail')
            if attempt < max_attempts:
                try:
                    if safe_click(driver, 'query_ticket', by=By.ID, timeout=5):
                        time.sleep(random.uniform(*refresh_interval))
                    else:
                        driver.refresh()
                        time.sleep(random.uniform(*refresh_interval))
                except Exception:
                    pass
    return '没抢到，可惜~'

In [30]:
if __name__ == "__main__":
    # 获取参数（优先 config.json；否则交互输入）
    params = get_params()

    start_position = params["start_position"]
    reach_position = params["reach_position"]
    start_time = params["start_date"]
    ticket_type = params["ticket_type"]  # 'adult' | 'student'
    target_train_number = params["target_train_number"]
    booking_start_time = params["booking_start_time"]
    seat_class_priority = params.get("seat_class_priority", ["二等座", "一等座", "商务座"])
    seat_preference = params.get("seat_preference", "过道")
    allow_reserve = params.get("allow_reserve", False)
    retry_cfg = params.get("retry", {"max_attempts": 30, "refresh_interval": [0.5, 1.5]})
    burst_cfg = params.get("burst", {"enable": True, "duration_sec": 2.0, "min_interval": 0.08, "max_interval": 0.16})
    warmup_seconds = float(params.get("warmup_seconds", 10))

    print(f"抢票任务设置: {start_position} -> {reach_position} {start_time} 车次={target_train_number}")
    print(f"席别优先: {seat_class_priority} | 选座偏好: {seat_preference} | 候补: {allow_reserve}")
    print(f"抢票开始时间: {booking_start_time}")
    print("请扫码登录，需要提前开启手机12306软件的扫码工具")

    from selenium.webdriver.edge.options import Options

    edge_options = Options()
    edge_options.add_experimental_option("detach", True)
    edge_options.add_argument("--disable-blink-features=AutomationControlled")
    edge_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.3485.54")
    driver = webdriver.Edge(options=edge_options)
    driver.get('https://www.12306.cn')
    driver.maximize_window()

    try:
        login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, 'J-btn-login'))
        )
        login_button.click()

    except Exception as e:
        print(f"点击登录按钮失败：{str(e)}")
        driver.quit()
        raise

    try:
        scan_login_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, "//a[text()='扫码登录']"))
        )
        scan_login_button.click()
        print("已切换到扫码登录，请用手机12306扫码...")

    except Exception as e:
        print(f"点击扫码登录按钮失败：{str(e)}")
        driver.quit()
        raise

    login_success = False
    print("等待扫码登录...")
    for _ in range(40):
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//a[text()='个人中心']"))
            )
            login_success = True
            break
        except Exception:
            time.sleep(1)

    if not login_success:
        print("登录超时或未成功，请重新尝试")
        driver.quit()
        raise SystemExit(1)

    try:
        ticket_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'link_for_ticket'))
        )
        ticket_link.click()
        print("已进入购票页面")

    except Exception as e:
        print(f"进入购票页面失败：{str(e)}")
        driver.quit()
        raise

    # 出发地/目的地/日期
    try:
        safe_click(driver, '#fromStationText')
        safe_send_keys(driver, '#fromStationText', start_position)
        safe_click(driver, '#citem_0 > span:nth-child(1)')
    except Exception as e:
        print(f"操作出发地失败: {e}")
        raise

    try:
        safe_click(driver, '#toStationText')
        safe_send_keys(driver, '#toStationText', reach_position)
        safe_click(driver, '#citem_0 > span:nth-child(1)')
    except Exception as e:
        print(f"操作目的地失败: {e}")
        raise

    try:
        safe_click(driver, '#train_date')
        safe_send_keys(driver, '#train_date', start_time)
        # 触发收起日历
        try:
            driver.find_element(By.CLASS_NAME, 'cal').click()
        except Exception:
            pass
    except Exception as e:
        print(f"时间输入框操作失败：{str(e)}")
        raise

    # 票型
    try:
        if ticket_type == 'student':
            safe_click(driver, 'sf2', by=By.ID)
            print('已选择学生票')
        else:
            safe_click(driver, 'sf1', by=By.ID)
            print('已选择普通票')
    except Exception as e:
        print(f"票种选择失败：{str(e)}")
        raise

    # 等待到开售时间，支持预热与连点
    try:
        start_dt = datetime.strptime(booking_start_time, "%Y-%m-%d %H:%M:%S")
        now = datetime.now()
        if now < start_dt:
            precisely_wait_until(start_dt, warmup_seconds=warmup_seconds)
        else:
            print('已过开售时间，直接开始抢票')

        if burst_cfg.get('enable', True):
            click_query_burst(
                driver,
                duration_sec=float(burst_cfg.get('duration_sec', 2.0)),
                min_interval=float(burst_cfg.get('min_interval', 0.08)),
                max_interval=float(burst_cfg.get('max_interval', 0.16)),
            )
        else:
            safe_click(driver, 'query_ticket', by=By.ID)
            time.sleep(1)
    except Exception as e:
        print(f"到点前准备/连点失败：{e}")

    print(f"开始尝试预订 {target_train_number} 次列车（带席别过滤）...")
    result = retry_book_ticket_with_filters(
        driver=driver,
        train_no=target_train_number,
        seat_priority=seat_class_priority,
        allow_reserve=allow_reserve,
        max_attempts=int(retry_cfg.get('max_attempts', 30)),
        refresh_interval=tuple(retry_cfg.get('refresh_interval', [0.5, 1.5]))
    )
    print(result)

    # 订单页交互（乘客/提示/提交/选座）
    try:
        passenger_checkbox = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "normalPassenger_0"))
        )
        passenger_checkbox.click()
        print("已成功选择乘车人")
    except Exception as e:
        print(f"选择乘车人失败：{str(e)}")

    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "dialog_xsertcj_ok"))
        )
        confirm_button.click()
        print("已成功点击提示框的确认按钮")
    except Exception as e:
        print(f"点击确认按钮失败：{str(e)}")

    try:
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "submitOrder_id"))
        )
        submit_button.click()
        print("已成功点击提交订单按钮")
    except Exception as e:
        print(f"点击提交订单按钮失败：{str(e)}")

    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "qd_closeDefaultWarningWindowDialog_id"))
        )
        confirm_button.click()
        print("已成功点击提示框的确认按钮")
    except Exception as e:
        print(f"点击确认按钮失败：{str(e)}")

    time.sleep(1)
    try:
        select_seat_dynamically(driver, preferred_type=seat_preference)
    except Exception as e:
        print(f"选座步骤失败：{e}")
    time.sleep(1)

    try:
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "qr_submit_id"))
        )
        confirm_button.click()
    except Exception as e:
        print(f"点击最终确认失败：{str(e)}")

抢票任务设置: 龙川西 -> 河源东 2025-10-05 车次=G6541
席别优先: ['二等座'] | 选座偏好: 过道 | 候补: True
抢票开始时间: 2025-9-22 21:12:00
请扫码登录，需要提前开启手机12306软件的扫码工具
已切换到扫码登录，请用手机12306扫码...
等待扫码登录...
已切换到扫码登录，请用手机12306扫码...
等待扫码登录...
已进入购票页面
已进入购票页面
已选择普通票
已过开售时间，直接开始抢票
已选择普通票
已过开售时间，直接开始抢票
连点查询完成，次数=12
开始尝试预订 G6541 次列车（带席别过滤）...
第1次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211410.png, click_fail_20250922_211410.html
连点查询完成，次数=12
开始尝试预订 G6541 次列车（带席别过滤）...
第1次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211410.png, click_fail_20250922_211410.html
第2次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211411.png, click_fail_20250922_211411.html
第2次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211411.png, click_fail_20250922_211411.html
第3次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211412.png, click_fail_20250922_211412.html
第3次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211412.png, click_fail_20250922_211412.html
第4次尝试失败: Message: 未找到目标车次行

已保存快照: click_fail_20250922_211413.png, clic